In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from Clustering import Clustering
from DataNoiseAdder import DataNoiseAdder
from DatasetCorruptor import DatasetCorruptor
from DecisionTreeEnsemble import DecisionTreeEnsemble
from SyntheticDataGenerator import SyntheticDataGenerator
from EnsembleDiversity import EnsembleDiversity
from EnsembleMetrics import EnsembleMetrics

from utils import get_dataset, get_ensemble_preds_from_models, get_precision_recall_auc, auprc_threshs
from utils import plot_precision_recall, plot_aroc_at_curve, fitness_scatter
from utils import compute_metrics_in_buckets, flatten_df, compute_cluster_metrics
from utils import get_categorical_and_float_features
from utils import get_clusters_dict, make_noise_preds

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

### Experiment Args

In [2]:
args = {}
args['ntrls'] = 10
args['ensemble_size'] = 10
args['dataset_path'] = "/Users/scottmerrill/Documents/UNC/Research/OOD-Ensembles/datasets"
args['dataset_name'] = 'heloc_tf' 

# Decision Tree/Model Pool Params
args['num_classifiers'] = 100
args['feature_fraction'] = 0.5
args['data_fraction'] = 0.8
args['max_depth'] = 10
args['min_samples_leaf'] = 5
args['random_state'] = 1
args['clusters_list'] = [3, 10]
args['shift_feature_count'] = 5

AUCTHRESHS = np.array([0.1, 0.2, 0.3, 0.4, 1. ])

In [3]:
x_train, y_train, x_val_id, y_val_id, x_val_ood, y_val_ood = get_dataset(args['dataset_path'] , args['dataset_name'])
num_features = x_train.shape[1]

### Building and Training Model Pool

In [4]:
model_pool = DecisionTreeEnsemble(args['num_classifiers'], 
                                  args['feature_fraction'],
                                  args['data_fraction'],
                                  args['max_depth'],
                                  args['min_samples_leaf'],
                                  args['random_state'])

# train model pool
model_pool.train(x_train, y_train)

### Caching Model Pool Predictions

In [5]:
model_pool_preds = model_pool.predict(x_val_ood)
model_pool_pred_probs = model_pool.predict_proba(x_val_ood)
mp_precision, mp_recall, mp_auc = get_precision_recall_auc(model_pool_pred_probs, y_val_ood, AUCTHRESHS)

### Caching Individual Model Predictions

In [6]:
model_pool.train_preds = model_pool.get_individual_predictions(x_train).T
model_pool.train_pred_probs = model_pool.get_individual_probabilities(x_train)

model_pool.val_id_preds = model_pool.get_individual_predictions(x_val_id).T
model_pool.val_id_pred_probs = model_pool.get_individual_probabilities(x_val_id)

model_pool.val_ood_preds = model_pool.get_individual_predictions(x_val_ood).T
model_pool.val_ood_pred_probs = model_pool.get_individual_probabilities(x_val_ood)

### Clustering Data and noise methods

In [7]:
all_clusters_dict = {}
all_clusters_dict['train_preds'], all_clusters_dict['val_id']  = get_clusters_dict(x_train, x_val_id, args['clusters_list'])
all_clusters_dict = make_noise_preds(x_train, y_train, x_val_id, model_pool, args['shift_feature_count'], args['clusters_list'], all_clusters_dict)

### Label Shift

In [8]:
y_train_flipped = DataNoiseAdder.label_flip(y_train)
y_val_flipped = DataNoiseAdder.label_flip(y_val_id)

### Random Search Loop

In [9]:
a = [x for x in dir(model_pool) if 'preds' in x]
b = [x for x in dir(model_pool) if 'pred_' in x]
a.remove('val_ood_preds')
b.remove('val_ood_pred_probs')
pred_attributes = [(a[i], b[i], a[i].split('_')[0]+'_'+a[i].split('_')[1]) for i in range(len(a))]

In [10]:
precisions_df = pd.DataFrame()
recalls_df = pd.DataFrame()
aucs_df = pd.DataFrame()
fitness_df = pd.DataFrame()

for trial in tqdm(range(args['ntrls'])):
    indices = np.random.choice(model_pool.num_classifiers, size=args['ensemble_size'], replace=True)

    # ood preds of sub-ensemble
    ood_preds, ood_pred_probs = get_ensemble_preds_from_models(model_pool.val_ood_pred_probs[indices])

    # save OOD precision/recalls seprately
    precision, recall, auc = get_precision_recall_auc(ood_pred_probs, y_val_ood, AUCTHRESHS)

    recalls_df = pd.concat([recalls_df, pd.DataFrame(recall)], axis=1)
    precisions_df = pd.concat([precisions_df, pd.DataFrame(precision)], axis=1)
    aucs_df = pd.concat([aucs_df, pd.DataFrame(auc)], axis=1)

    tmp = {'generation':trial,
              'ensemble_files':','.join(str(x) for x in indices)}
    cluster_metrics = pd.DataFrame()

    # Compute all Fitness Metrics
    for label_flip in [0, 1]:
        for pred_tuple in pred_attributes:
            preds_name, pred_prob_name, prefix_name = pred_tuple

            # get clustering associated with data transformation
            clusters_dict = all_clusters_dict[prefix_name]
            if 'train' in prefix_name:
                if label_flip:
                    Y = y_train_flipped
                    prefix_name = prefix_name + '_flip'
                else:
                    Y = y_train
                    
            else:
                if label_flip:
                    prefix_name = prefix_name + '_flip'
                    Y = y_val_flipped
                else:
                    Y = y_val_id

            model_pool_preds = getattr(model_pool, preds_name)
            model_pool_pred_probs = getattr(model_pool, pred_prob_name)

            model_preds = model_pool_preds[indices]
            model_pred_probs = model_pool_pred_probs[indices]

            # id val preds of sub-ensemble
            ensemble_preds, ensemble_pred_probs = get_ensemble_preds_from_models(model_pred_probs)
            metrics = EnsembleMetrics(Y, ensemble_preds, ensemble_pred_probs[:,1])
            diversity = EnsembleDiversity(Y, model_preds)

            tmp.update({f'{prefix_name}_acc':metrics.accuracy(),
                   f'{prefix_name}_auc':metrics.auc(),
                   f'{prefix_name}_prec':metrics.precision(),
                   f'{prefix_name}_rec':metrics.recall(),
                   f'{prefix_name}_f1':metrics.f1(),
                   f'{prefix_name}_mae':metrics.mean_absolute_error(),
                   f'{prefix_name}_mse':metrics.mean_squared_error(),
                   f'{prefix_name}_logloss':metrics.log_loss(),

                   # diversity
                   f'{prefix_name}_q_statistic':np.mean(diversity.q_statistic()),
                   f'{prefix_name}_correlation_coefficient':np.mean(diversity.correlation_coefficient()),
                   f'{prefix_name}_entropy':np.mean(diversity.entropy()),
                   f'{prefix_name}_diversity_measure':diversity.diversity_measure(),
                   f'{prefix_name}_hamming_distance':np.mean(diversity.hamming_distance()),
                   f'{prefix_name}_error_rate':np.mean(diversity.error_rate()),
                   f'{prefix_name}_auc':np.mean(diversity.auc()),
                   f'{prefix_name}_brier_score':np.mean(diversity.brier_score()),
                   f'{prefix_name}_ensemble_variance':np.mean(diversity.ensemble_variance()),
                  })

            # compute cluster metrics
            tmp_cluster = compute_cluster_metrics(clusters_dict, ensemble_preds, model_preds, model_pred_probs, Y)
            col_names = [prefix_name + '_' + x for x in tmp_cluster.columns]
            col_names = [name.replace('_val_acc', '') for name in col_names]
            col_names = [name.replace('_train_acc', '') for name in col_names]
            tmp_cluster.columns = col_names

            cluster_metrics = pd.concat([cluster_metrics, tmp_cluster], axis=1)

    raw_metrics = pd.DataFrame([tmp])    
    tmp = pd.concat([raw_metrics, cluster_metrics], axis=1)
    fitness_df = pd.concat([fitness_df, tmp])
    #precisions_df.to_csv(save_path+'/precisions_df.csv', index=False)
    #recalls_df.to_csv(save_path+'/recalls_df.csv', index=False)
    #aucs_df.to_csv(save_path+'/aucs_df.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
pd.set_option('display.max_columns', None)
fitness_df

generation                 ensemble_files  train_boundaryshift_acc  \
0           0   49,90,80,84,27,0,16,62,23,58                 0.696926   
0           1   48,72,70,21,64,83,8,65,15,49                 0.636408   
0           2   92,67,50,80,49,7,34,53,33,72                 0.663520   
0           3  41,97,81,69,71,89,37,49,69,73                 0.653353   
0           4  56,15,18,81,95,28,75,30,44,68                 0.640765   
0           5  70,11,86,66,52,72,89,35,75,20                 0.652869   
0           6   87,94,96,27,25,34,69,17,94,9                 0.652626   
0           7    60,83,49,58,8,7,23,45,25,12                 0.673929   
0           8  30,57,17,60,33,90,31,86,25,21                 0.657468   
0           9    80,18,5,66,17,5,60,18,53,49                 0.652384   

   train_boundaryshift_auc  train_boundaryshift_prec  train_boundaryshift_rec  \
0                 0.613221                  0.724794                 0.582866   
0                 0.585295                  0.664046                 0.473738   
0                 0.598860                  0.683601                 0.542070   
0                 0.595752                  0.686662                 0.496175   
0                 0.587550                  0.685892                 0.448751   
0                 0.592669                  0.685694                 0.496175   
0                 0.595893                  0.673483                 0.520653   
0                 0.603692                  0.719286                 0.513514   
0                 0.595471                  0.688017                 0.509434   
0                 0.595743                  0.686038                 0.493626   

   train_boundaryshift_f1  train_boundaryshift_mae  train_boundaryshift_mse  \
0                0.646128                 0.303074                 0.303074   
0                0.552976                 0.363592                 0.363592   
0                0.604664                 0.336480                 0.336480   
0                0.576081                 0.346647                 0.346647   
0                0.542540                 0.359235                 0.359235   
0                0.575740                 0.347131                 0.347131   
0                0.587288                 0.347374                 0.347374   
0                0.599226                 0.326071                 0.326071   
0                0.585409                 0.342532                 0.342532   
0                0.574140                 0.347616                 0.347616   

   train_boundaryshift_logloss  train_boundaryshift_q_statistic  \
0                     0.797100                         0.146378   
0                     0.887510                         0.138285   
0                     0.861151                         0.147840   
0                     1.046248                         0.152888   
0                     0.827425                         0.133716   
0                     0.757817                         0.132834   
0                     0.964483                         0.161548   
0                     0.921263                         0.151928   
0                     0.762736                         0.127937   
0                     1.140492                         0.161301   

   train_boundaryshift_correlation_coefficient  train_boundaryshift_entropy  \
0                                     0.076083                     8.166450   
0                                     0.071506                     8.156009   
0                                     0.076685                     8.161577   
0                                     0.083060                     8.126418   
0                                     0.068673                     8.144873   
0                                     0.068451                     8.168059   
0                                     0.087799                     8.139204   
0                                     0.078001                     

### Plot Results

In [ ]:
fitness_df = fitness_df.reset_index(drop=True)
best_fitness_index = {i+1:index for i,index in enumerate(fitness_df.nlargest(3, 'val_upscaleshift_meanshift_val_error_rate_mean').index)}

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

plot_precision_recall(precisions_df, recalls_df, mp_precision, mp_recall, best_fitness_index, ax=axs[0])
plot_aroc_at_curve(AUCTHRESHS, aucs_df, mp_auc, best_fitness_index, ax=axs[1])

### Fitness Function Diagnosis


In [ ]:
cols = ['train_boundaryshift_acc', 'val_upscaleshift_meanshift_val_error_rate_mean']
fig, axs = plt.subplots(1, len(cols), figsize=(12, 6))

for idx, col in enumerate(cols):
    # Plot AUC scatter for different fitness columns
    fitness_scatter(fitness_df, aucs_df, col, ax=axs[idx])
plt.tight_layout(pad=2)

In [ ]:
pd.set_option('display.max_columns',None)
fitness_df.head()